
# 🧪 NovaMind — AI-Powered Marketing Content Pipeline 




In [92]:

import os
import json
import time
import textwrap
import random
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, Any, List, Optional, Tuple
from datetime import datetime
from pathlib import Path

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-your-api-key-here").strip()
# OPENAI_API_KEY =

ARTIFACTS_DIR = Path("artifacts")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

PERSONAS = ["founders", "creatives", "ops"]

def slugify(s: str) -> str:
    s = s.lower().strip().replace(" ", "-")
    allowed = set("abcdefghijklmnopqrstuvwxyz0123456789-")
    return "".join(ch for ch in s if ch in allowed)

# Basic sanity check
if not OPENAI_API_KEY or OPENAI_API_KEY.startswith("sk-your-"):
    print("""⚠️ You haven't set a real API key yet.
Go to the config cell and set OPENAI_API_KEY to your actual key or export it as an environment variable.
""")



# 🤖OPENAI API 
This client always uses the OpenAI API. If the call fails, it returns a clear fallback message.


In [64]:
# !pip install openai

In [95]:

from openai import OpenAI

class AIClient:
    def __init__(self, api_key: str):
        if not api_key or api_key.startswith("sk-your-"):
            raise RuntimeError("OpenAI API key missing. Set OPENAI_API_KEY in the config cell.")
        try:
            self._client = OpenAI(api_key=api_key)
            print("✅ OpenAI client initialized.")
        except Exception as e:
            raise RuntimeError(f"❌ Failed to initialize OpenAI client: {e}")

    def generate(self, prompt: str, max_tokens: int = 800, temperature: float = 0.7) -> str:
        try:
            resp = self._client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a concise, practical marketing copy generator for AI-powered automation tools."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            return textwrap.fill(
                f"[Fallback] OpenAI error: {e}. Please verify your API key / network and re-run.", width=88
            )

    def summarize(self, prompt: str, max_tokens: int = 400, temperature: float = 0.5) -> str:
        return self.generate(prompt, max_tokens=max_tokens, temperature=temperature)


ai = AIClient(api_key=OPENAI_API_KEY)


✅ OpenAI client initialized.



## ✍️ 1. Content Generation
Enter a topic → produce a structured blog and 3 persona-tailored newsletters. Artifacts are saved to `artifacts/`.


In [119]:

BLOG_TEMPLATE = """# {title}

**TL;DR**  
{tldr}

## Why this matters
{section1}

## What to try this week
{section2}

## A tiny case study
{section3}

## Checklist
- {check1}
- {check2}
- {check3}

"""
#This defines how each audience segment thinks —it’s like giving OpenAI personality cues for tailoring tone and focus.
PERSONA_ANGLE = {
    "founders": "Emphasize ROI, growth levers, and time/cost savings. Be crisp; show impact.",
    "creatives": "Inspire with visual-first workflows and quicker iteration. Mention mood boards and asset hygiene.",
    "ops": "Stress reliability, integrations, and reduced manual error. Highlight checklists and change management.",
}

def generate_blog_and_newsletters(topic: str, brand_voice: str = "practical, optimistic, no fluff") -> Dict[str, Any]:
    title = f"{topic.strip().title()}: From Idea To Impact In A Week"

# Ask OpenAI for the body - builds a prompt for open AI and AI client (ai.generate),asking it to produce the main blog content in your brand's voice. 
    blog_prompt = f"""Write a 400–600 word blog post in a {brand_voice} voice on the topic '{topic}'.
    Audience: small creative agencies. Provide a TL;DR, three practical sections, and a short checklist."""
    blog_body = ai.generate(blog_prompt, max_tokens=900)

'''
    # Scaffold adds predictable structure above the generated body
    You add your own fixed, high-quality intro paragraphs and bullet points for consistency across campaigns.
    This makes your output hybrid: part static framework, part generative creativity.'''

    tldr = "Automation patterns small teams can apply in under an hour to unlock speed and focus."
    section1 = "Automation reduces copy-paste toil and context switching, letting teams ship more with less stress."
    section2 = "Try a templated brief, one-click project setup, and auto-routing approvals tied to due dates."
    section3 = "A 5-person studio cut handoff time by 28% by standardizing briefs and automating asset delivery."
    check1, check2, check3 = "Standardize briefs", "Auto-create tasks from templates", "Route approvals with due dates"

    blog_markdown = BLOG_TEMPLATE.format(
        title=title, tldr=tldr, section1=section1, section2=section2, section3=section3,
        check1=check1, check2=check2, check3=check3
    ) + "\n\n" + blog_body

    newsletters = {}
    for p in PERSONAS:
        persona_prompt = f"""Condense the blog on '{topic}' into a 120–160 word newsletter for persona '{p}'.
        Angle: {PERSONA_ANGLE[p]}. Include a compelling subject line and a CTA to 'Read the full blog'."""
        newsletters[p] = ai.generate(persona_prompt, max_tokens=400)

    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    slug = slugify(topic)
    out_dir = ARTIFACTS_DIR / f"campaign_{ts}_{slug}"
    out_dir.mkdir(parents=True, exist_ok=True)

    payload = {
        "topic": topic,
        "title": title,
        "created_at": ts,
        "personas": PERSONAS,
        "blog_markdown": blog_markdown,
        "newsletters": newsletters,
    }
    (out_dir / "content.json").write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")
    (out_dir / "blog.md").write_text(blog_markdown, encoding="utf-8")
    #this part gives json to later synced to a CRM as well as a markdown file for human-readable structure 
    
    for p in PERSONAS:
        (out_dir / f"newsletter_{p}.md").write_text(newsletters[p], encoding="utf-8")

    print(f"Saved content to: {out_dir}")
    return payload #Returns a Python dictionary for the next pipeline stages (e.g., CRM distribution or dashboard).

# ▶️ Run content generation
TOPIC = "AI in creative automation"
content = generate_blog_and_newsletters(TOPIC)
content['title']


Saved content to: artifacts/campaign_20251019-220708_ai-in-creative-automation


'Ai In Creative Automation: From Idea To Impact In A Week'


## 📇 2. Connect CRM API -Zoho and Distribute 

In [113]:
import requests

# STEP 1 — fill in your details
CLIENT_ID = "1000.0PX8CMAL0JUZ4FNS85W7B5BAJJTH9A"
CLIENT_SECRET = "ba667cb375048f83799863848e2ddfb05a7d32fdaa"
GRANT_TOKEN = "1000.b292049fd94b4005624ea8fb8e6f91d0.ef2e2328bc61c4be7b3bf47cc8d31970"  
# STEP 2 — make the request
token_url = "https://accounts.zoho.com/oauth/v2/token"

params = {
    "grant_type": "authorization_code",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "redirect_uri": "https://www.zoho.com",  # dummy for self-client
    "code": GRANT_TOKEN
}

response = requests.post(token_url, params=params)

# STEP 3 — print the access token
print("Status:", response.status_code)
print(response.json())



Status: 200
{'access_token': '1000.5e68f97146f1e9e3c0278448dec49ca4.9689e033d71385958e6cfa938b9709e7', 'refresh_token': '1000.b814fdc375a414fd73b8575f150aeb3b.1a6b2d5e5cc792dc75e046e863ba641e', 'scope': 'ZohoCRM.modules.ALL', 'api_domain': 'https://www.zohoapis.com', 'token_type': 'Bearer', 'expires_in': 3600}


 ✅ the next block : it reads the existing contacts and also create new contact if any

In [115]:

import requests, json

ACCESS_TOKEN = "1000.5e68f97146f1e9e3c0278448dec49ca4.9689e033d71385958e6cfa938b9709e7"

# Basic setup
headers = {
    "Authorization": f"Zoho-oauthtoken {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Example 1 — Read contacts (list first 3)
url = "https://www.zohoapis.com/crm/v2/Contacts?per_page=3"
r = requests.get(url, headers=headers)
print("Contacts →", r.status_code)
print(json.dumps(r.json(), indent=2)[:500])

# Example 2 — Create a new contact
new_contact = {
    "data": [
        {
            "Last_Name": "C",
            "First_Name": "Haley",
            "Email": "haley@example.com",
            "Description": "Added via NovaMind AI Marketing Pipeline demo"
        }
    ]
}

r2 = requests.post("https://www.zohoapis.com/crm/v2/Contacts", headers=headers, json=new_contact)
print("\nCreate Contact →", r2.status_code)
print(r2.text[:300])

Contacts → 200
{
  "data": [
    {
      "Owner": {
        "name": "Haley Chen",
        "id": "7058666000000575001",
        "email": "shuxuanhchen@gmail.com"
      },
      "Email": "krismarrier@noemail.invalid",
      "$currency_symbol": "$",
      "$field_states": null,
      "Other_Phone": null,
      "Mailing_State": "MD",
      "Other_State": null,
      "Other_Country": null,
      "Last_Activity_Time": "2025-10-19T21:49:10-04:00",
      "Department": "Engineering",
      "$state": "save",
      "Unsu

Create Contact → 201
{"data":[{"code":"SUCCESS","details":{"Modified_Time":"2025-10-19T22:00:32-04:00","Modified_By":{"name":"Haley Chen","id":"7058666000000575001"},"Created_Time":"2025-10-19T22:00:32-04:00","id":"7058666000000597001","Created_By":{"name":"Haley Chen","id":"7058666000000575001"}},"message":"record adde


### 📨Newsletter Distribution - MOCK 
---------------------------------------------------------
Requirements satisfied:

- Tag or segment contacts by persona
- Log each campaign (blog title, newsletter ID, send date)

In [125]:


from datetime import datetime

ACCESS_TOKEN = "1000.5e68f97146f1e9e3c0278448dec49ca4.9689e033d71385958e6cfa938b9709e7"
API_BASE = "https://www.zohoapis.com/crm/v2"

headers = {
    "Authorization": f"Zoho-oauthtoken {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# --------------------------------------------
# Create or update mock contacts for each persona
# --------------------------------------------
PERSONAS = ["founders", "creatives", "ops"]
contacts = []

for persona in PERSONAS:
    for i in range(3):  # a few mock entries per persona
        contact = {
            "Last_Name": f"{persona.capitalize()}_{i}",
            "First_Name": "Demo",
            "Email": f"{persona}{i}@novamind.example.com",
            "Description": f"Segment: {persona.capitalize()}",
            "Lead_Source": "AI Newsletter Signup"
        }
        contacts.append(contact)

payload = {"data": contacts} # bundle all contacts into a Zoho-compatible payload

# ✅ Use "upsert" so existing emails get updated instead of causing duplicate errors
r = requests.post(f"{API_BASE}/Contacts/upsert", headers=headers, json=payload)
print("📇 Create or update contacts:", r.status_code)
print(r.text[:400])


# --------------------------------------------
#  Log campaign info into CRM (using Notes module)
# --------------------------------------------
campaign_title = "AI in Creative Automation"
send_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 🔍 Get one existing contact ID to attach the Note to
contact_list = requests.get(f"{API_BASE}/Contacts?per_page=1", headers=headers).json()
parent_id = contact_list["data"][0]["id"]

#prepare and send the Note: ✅ This creates a CRM Note attached to that contact (open Zoho → Contacts → Notes) 
campaign_log = {
    "data": [
        {
            "Note_Title": f"Campaign Log - {campaign_title}",
            "Note_Content": f"Campaign sent on {send_date} to segments: {', '.join(PERSONAS)}",
            "Parent_Id": parent_id,      # required field to attach the note
            "se_module": "Contacts"      # tells Zoho which module to attach to
        }
    ]
}

r2 = requests.post(f"{API_BASE}/Notes", headers=headers, json=campaign_log)
print("\n🗒️  Log campaign:", r2.status_code)
print(r2.text[:400])

# --------------------------------------------
#  Simulate sending newsletters - Simulate newsletter sending
# --------------------------------------------
for persona in PERSONAS:
    print(f"✉️  Sending newsletter to {persona.capitalize()} segment...")
    # Here you'd normally call an email API or CRM automation trigger

📇 Create or update contacts: 200
{"data":[{"code":"SUCCESS","duplicate_field":"Email","action":"update","details":{"Modified_Time":"2025-10-19T22:09:58-04:00","Modified_By":{"name":"Haley Chen","id":"7058666000000575001"},"Created_Time":"2025-10-19T22:02:39-04:00","id":"7058666000000599001","Created_By":{"name":"Haley Chen","id":"7058666000000575001"}},"message":"record updated","status":"success"},{"code":"SUCCESS","duplicate_fi

🗒️  Log campaign: 201
{"data":[{"code":"SUCCESS","details":{"Modified_Time":"2025-10-19T22:09:59-04:00","Modified_By":{"name":"Haley Chen","id":"7058666000000575001"},"Created_Time":"2025-10-19T22:09:59-04:00","id":"7058666000000606001","Created_By":{"name":"Haley Chen","id":"7058666000000575001"}},"message":"record added","status":"success"}]}
✉️  Sending newsletter to Founders segment...
✉️  Sending newsletter to Creatives segment...
✉️  Sending newsletter to Ops segment...



# 3. 📈 Performance Logging & Analysis



In [149]:
# 📊 PERFORMANCE LOGGING & ANALYSIS
# ---------------------------------------------------------
# Focus: Core 3 metrics — Open Rate, CTR, Conversion Rate
# ---------------------------------------------------------
# - Simulate newsletter performance metrics
# - Store data historically
# - Generate structured AI-powered analysis (Performance + Conversion + Recommendation)

import pandas as pd
import numpy as np
from datetime import datetime
import json
import os

# ---------------------------------------------------------
# 1️⃣ Simulate newsletter performance metrics
# ---------------------------------------------------------
PERSONAS = ["founders", "creatives", "ops"]

np.random.seed(42)  # reproducible randomness

performance_data = []
for persona in PERSONAS:
    record = {
        "persona": persona,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        # Core metrics
        "open_rate": round(np.random.uniform(0.35, 0.75), 3),
        "ctr": round(np.random.uniform(0.05, 0.25), 3),
        "conversion_rate": round(np.random.uniform(0.01, 0.10), 3)
    }
    performance_data.append(record)

df = pd.DataFrame(performance_data)
print("📈 Simulated Newsletter Performance (3 Key Metrics):\n")
print(df)

# ---------------------------------------------------------
# 2️⃣ Store data for historical comparison
# ---------------------------------------------------------
os.makedirs("artifacts", exist_ok=True)
perf_file = "artifacts/performance_history.csv"

if os.path.exists(perf_file):
    old_df = pd.read_csv(perf_file)
    df = pd.concat([old_df, df], ignore_index=True)

df.to_csv(perf_file, index=False)
print(f"\n💾 Performance data saved to {perf_file}")

# ---------------------------------------------------------
# 3️⃣ AI-powered structured performance summary
# ---------------------------------------------------------
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)  # Replace with env var

# Structured prompt for consistent, sectioned output
summary_prompt = f"""
You are a senior marketing data analyst reviewing NovaMind's newsletter results.

Data summary (Open Rate, CTR, Conversion Rate):
{df.tail(3).to_string(index=False)}

Write a structured analysis with clear sections:

### Performance Overview
Summarize which persona performed best overall and what patterns stand out.

### Conversion Analysis
Explain which audience segment shows strongest conversion intent and hypothesize why (content, tone, offer relevance).

### Actionable Recommendation
Give ONE clear, data-driven suggestion for next week's newsletter content — focused on improving engagement and conversion.

Keep insights concise (≤200 words total) and professional.
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a data-driven marketing strategist providing structured, executive-style summaries."},
        {"role": "user", "content": summary_prompt}
    ],
    max_tokens=300,
    temperature=0.7
)

summary = response.choices[0].message.content.strip()
print("\n🧠 AI-Powered Marketing Summary:\n")
print(summary)

# ---------------------------------------------------------
# 4️⃣ Save structured summary for historical tracking
# ---------------------------------------------------------
summary_entry = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "summary": summary
}

with open("artifacts/performance_summaries.json", "a", encoding="utf-8") as f:
    json.dump(summary_entry, f, ensure_ascii=False)
    f.write("\n")

print("\n🗂️ Summary saved to artifacts/performance_summaries.json")

📈 Simulated Newsletter Performance (3 Key Metrics):

     persona            timestamp  open_rate    ctr  conversion_rate
0   founders  2025-10-21 13:40:25      0.500  0.240            0.076
1  creatives  2025-10-21 13:40:25      0.589  0.081            0.024
2        ops  2025-10-21 13:40:25      0.373  0.223            0.064

💾 Performance data saved to artifacts/performance_history.csv

🧠 AI-Powered Marketing Summary:

### Performance Overview
Among the three personas, the **creatives** segment demonstrated the highest open rate at **58.9%**, indicating strong initial engagement. The **founders** followed with a **50.0%** open rate, while **ops** had the lowest at **37.3%**. Notably, the **founders** achieved the highest click-through rate (CTR) of **24.0%**, suggesting effective content resonance despite lower overall engagement.

### Conversion Analysis
The **founders** segment exhibited the strongest conversion rate at **7.6%**, reflecting a solid intent to act on the content. Th